In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import os

from sklearn.model_selection import train_test_split

import tensorflow as tf

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [2]:
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/1.0)



In [3]:
train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=125)

In [4]:
def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.image.resize(img, [28,28])
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.image.resize(img, [28,28])
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

2024-03-25 10:45:56.108979: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-25 10:45:56.109003: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-25 10:45:56.109010: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-25 10:45:56.109041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 10:45:56.109058: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(16)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [6]:
# building the model
cnn_model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=[28, 28, 3], axis=1),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='relu'),
])

# compiling the model with 'adam' optimizer,
# "mean_squared_error" loss, and RootMeanSquaredError metric
cnn_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [7]:
# fitting the model with Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

history = cnn_model.fit(
    train_dataset,
    batch_size = 16,
    validation_data = val_dataset,
    epochs=100,
    callbacks=[callback]
)

Epoch 1/100


2024-03-25 10:45:57.145415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


496/496 [==============================] - 17s 29ms/step - loss: 485.3910 - root_mean_squared_error: 22.0316 - val_loss: 428.4449 - val_root_mean_squared_error: 20.6989
Epoch 2/100
496/496 [==============================] - 8s 16ms/step - loss: 469.6939 - root_mean_squared_error: 21.6724 - val_loss: 433.2006 - val_root_mean_squared_error: 20.8135
Epoch 3/100
496/496 [==============================] - 8s 15ms/step - loss: 487.0179 - root_mean_squared_error: 22.0685 - val_loss: 450.3403 - val_root_mean_squared_error: 21.2212
Epoch 4/100
496/496 [==============================] - 8s 15ms/step - loss: 470.9978 - root_mean_squared_error: 21.7025 - val_loss: 407.5622 - val_root_mean_squared_error: 20.1882
Epoch 5/100
496/496 [==============================] - 8s 15ms/step - loss: 474.9446 - root_mean_squared_error: 21.7932 - val_loss: 406.5799 - val_root_mean_squared_error: 20.1638
Epoch 6/100
496/496 [==============================] - 8s 15ms/step - loss: 457.9887 - root_mean_squared_error: